In [3]:



import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon

# Loading the  reduced samples
collisions = pd.read_csv("Motor_Vehicle_Collisions_Crashes.csv", low_memory=False).head(500)
arrests = pd.read_csv("NYP_ArrestData.csv", low_memory=False).head(300)

# Dropping  NaNs in location columns
arrests = arrests.dropna(subset=['Latitude', 'Longitude'])
collisions = collisions.dropna(subset=['BOROUGH'])

# Converting  arrests to GeoDataFrame
geometry = [Point(xy) for xy in zip(arrests['Longitude'], arrests['Latitude'])]
arrests_gdf = gpd.GeoDataFrame(arrests, geometry=geometry, crs='EPSG:4326')

# Rough bounding boxes for boroughs
boroughs_data = {
    "BOROUGH": ["MANHATTAN", "BROOKLYN", "QUEENS", "BRONX", "STATEN ISLAND"],
    "geometry": [
        Polygon([(-74.02, 40.7), (-73.93, 40.7), (-73.93, 40.88), (-74.02, 40.88)]),
        Polygon([(-74.05, 40.65), (-73.85, 40.65), (-73.85, 40.72), (-74.05, 40.72)]),
        Polygon([(-73.95, 40.68), (-73.7, 40.68), (-73.7, 40.8), (-73.95, 40.8)]),
        Polygon([(-73.93, 40.8), (-73.85, 40.8), (-73.85, 40.91), (-73.93, 40.91)]),
        Polygon([(-74.25, 40.5), (-74.05, 40.5), (-74.05, 40.65), (-74.25, 40.65)])
    ]
}
boroughs_gdf = gpd.GeoDataFrame(boroughs_data, crs='EPSG:4326')

# Spatial join to get boroughs in arrests
arrests_mapped = gpd.sjoin(arrests_gdf, boroughs_gdf, how='left', predicate='within')
arrests_mapped = arrests_mapped.drop(columns=['geometry', 'index_right'])

# Join with collision data on borough's and final merged file
merged = pd.merge(collisions, arrests_mapped, on='BOROUGH', how='inner')
merged.to_csv("merged_arrest_collision.csv", index=False)

print(" Merged file :")
print(merged.head())




 Merged file :
   CRASH DATE CRASH TIME   BOROUGH ZIP CODE  LATITUDE  LONGITUDE  \
0  11/01/2023       1:29  BROOKLYN    11230  40.62179 -73.970024   
1  11/01/2023       1:29  BROOKLYN    11230  40.62179 -73.970024   
2  11/01/2023       1:29  BROOKLYN    11230  40.62179 -73.970024   
3  11/01/2023       1:29  BROOKLYN    11230  40.62179 -73.970024   
4  11/01/2023       1:29  BROOKLYN    11230  40.62179 -73.970024   

                 LOCATION ON STREET NAME CROSS STREET NAME OFF STREET NAME  \
0  (40.62179, -73.970024)  OCEAN PARKWAY          AVENUE K             NaN   
1  (40.62179, -73.970024)  OCEAN PARKWAY          AVENUE K             NaN   
2  (40.62179, -73.970024)  OCEAN PARKWAY          AVENUE K             NaN   
3  (40.62179, -73.970024)  OCEAN PARKWAY          AVENUE K             NaN   
4  (40.62179, -73.970024)  OCEAN PARKWAY          AVENUE K             NaN   

   ...  ARREST_PRECINCT  JURISDICTION_CODE  AGE_GROUP  PERP_SEX  \
0  ...                5                 

In [4]:
# Loading the  datasets
economy_df = pd.read_excel("economy.xlsx")
socio_df = pd.read_excel("socio.xlsx")
# print("economy_df", economy_df.head())
# print("socio_df",socio_df.head())
# Merge on GeoID with suffixes to distinguish overlapping column names
merged_df = pd.merge(socio_df, economy_df, on='GeoID', suffixes=('_socio', '_econ'))

# Drop the  duplicated key columns from the economy dataset
merged_df.drop(columns=['GeoType_econ', 'GeogName_econ'], inplace=True, errors='ignore')

# Renaming the socio dataset to general names
merged_df.rename(columns={
    'GeoType_socio': 'GeoType',
    'GeogName_socio': 'GeogName'
}, inplace=True)

# Reorder to place key identifiers at the front
front_cols = ['GeoType', 'GeogName', 'GeoID']
other_cols = [col for col in merged_df.columns if col not in front_cols]
merged_df = merged_df[front_cols + other_cols]

# Removing the  columns with zero variance
merged_df = merged_df.loc[:, merged_df.nunique() > 1]

# final merged file
merged_df.to_csv("merged_socio_economic.csv", index=False)
merged_df.head()

,GeogName,GeoID,HH1E,HH1M,HH1C,Fam1E,Fam1M,Fam1C,Fam1P,Fam1Z,...,Pv400t499E,Pv400t499M,Pv400t499C,Pv400t499P,Pv400t499Z,Pv500plE,Pv500plM,Pv500plC,Pv500plP,Pv500plZ
0,City Council District 01,1,79496,2419,1.9,35833,1963,3.3,45.1,2.1,...,10744,1168,6.6,6.6,0.7,83570,3536,2.6,51.2,1.5
1,City Council District 02,2,82618,2187,1.6,25974,1553,3.6,31.4,1.7,...,8017,949,7.2,5.2,0.6,80121,3619,2.7,52.4,1.7
2,City Council District 03,3,90007,2137,1.4,25670,1507,3.6,28.5,1.5,...,9587,1104,7.0,6.3,0.7,89782,3444,2.3,59.3,1.6
3,City Council District 04,4,83689,2249,1.6,33572,1735,3.1,40.1,1.8,...,10529,1658,9.6,6.7,1.0,112354,3987,2.2,71.3,1.0
4,City Council District 05,5,84081,2562,1.9,35929,1922,3.3,42.7,1.9,...,9633,1261,8.0,6.1,0.8,105105,4330,2.5,66.2,1.5


In [5]:

# Loading datasets
socio_econ_df = pd.read_csv("merged_socio_economic.csv")
arrest_collision_df = pd.read_csv("merged_arrest_collision.csv")

#  Mapping City Council Districts to Boroughs
district_to_borough = {
    'MANHATTAN': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'BRONX': [10, 11, 12, 13, 14, 15, 16, 17, 18],
    'QUEENS': [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    'BROOKLYN': [31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45],
    'STATEN ISLAND': [46, 47, 48, 49, 50, 51]
}

district_borough_map = {}
for borough, districts in district_to_borough.items():
    for dist in districts:
        district_borough_map[f"City Council District {dist:02d}"] = borough

socio_econ_df['BOROUGH'] = socio_econ_df['GeogName'].map(district_borough_map)
socio_econ_df = socio_econ_df.dropna(subset=['BOROUGH'])

#  Aggregating socio-economic data by borough
socio_by_borough = socio_econ_df.groupby('BOROUGH').mean(numeric_only=True).reset_index()

# Selecting socio-economic features of interest
keywords = ['Pop', 'Inc', 'Pov', 'HH', 'Fam']
socio_columns = ['BOROUGH'] + [col for col in socio_by_borough.columns if any(k in col for k in keywords)]
socio_features = socio_by_borough[socio_columns].copy()

#  Merging with arrest + collision dataset
final_df = arrest_collision_df.merge(
    socio_features,
    left_on='BOROUGH',
    right_on='BOROUGH',
    how='left'
)

# selecting the  Final column
final_columns = [
    'ARREST_BORO', 'BOROUGH', 'PERP_RACE', 'PERP_SEX', 'AGE_GROUP'
] + [col for col in socio_features.columns if col != 'BOROUGH']

final_dataset = final_df[final_columns].copy()

# Saving to CSV
final_dataset.to_csv("final_merged_crime_socio_dataset.csv", index=False)

# Displaying the  top rows
print(final_dataset.head())


  ARREST_BORO   BOROUGH       PERP_RACE PERP_SEX AGE_GROUP          HH1E  \
0           M  BROOKLYN           BLACK        M     25-44  61467.933333   
1           M  BROOKLYN           BLACK        M     25-44  61467.933333   
2           K  BROOKLYN           BLACK        M     25-44  61467.933333   
3           M  BROOKLYN  WHITE HISPANIC        F       65+  61467.933333   
4           M  BROOKLYN           BLACK        M     45-64  61467.933333   

          HH1M      HH1C    Fam1E   Fam1M  ...    FamBwPvM  FamBwPvC  \
0  1403.466667  1.386667  37046.8  1320.0  ...  641.933333      7.44   
1  1403.466667  1.386667  37046.8  1320.0  ...  641.933333      7.44   
2  1403.466667  1.386667  37046.8  1320.0  ...  641.933333      7.44   
3  1403.466667  1.386667  37046.8  1320.0  ...  641.933333      7.44   
4  1403.466667  1.386667  37046.8  1320.0  ...  641.933333      7.44   

    FamBwPvP  FamBwPvZ       PopPvU1E     PopPvU1M  PopPvU1C       PopPvU2E  \
0  15.033333  1.653333  167691.

In [8]:

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from collections import Counter

categorical_columns = final_dataset.select_dtypes(include='object').columns.tolist()
numerical_columns = final_dataset.select_dtypes(include=['int64', 'float64']).columns.tolist()


# FEATURE ENGINEERING
# One-hot encode categorical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_columns)
    ],
    remainder='passthrough'  # Keep numeric features as they are
)

# Applying  transformation
processed_data = preprocessor.fit_transform(final_dataset)

# Assigning  new feature names
encoded_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns)
all_feature_names = list(encoded_feature_names) + numerical_columns

# Creating final processed DataFrame
processed_df = pd.DataFrame(processed_data, columns=all_feature_names)

#  ADDRESSING IMBALANCED Data

label_encoder = LabelEncoder()
target = label_encoder.fit_transform(final_dataset['ARREST_BORO'])

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(processed_df, target, test_size=0.2, random_state=42)

# Applying  SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Output
print("Original Training Set Shape:", X_train.shape)
print("Resampled Training Set Shape:", X_resampled.shape)
print("Class Distribution After SMOTE:", Counter(y_resampled))



Original Training Set Shape: (14827, 260)
Resampled Training Set Shape: (25265, 260)
Class Distribution After SMOTE: Counter({2: 5053, 1: 5053, 3: 5053, 0: 5053, 4: 5053})
